In [48]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.linear_model import SGDClassifier
from nltk.corpus import stopwords
import nltk
from string import punctuation
from nltk import pos_tag
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import time
start = time.clock()
#
category = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware',
'comp.sys.mac.hardware', 'rec.autos',
'rec.motorcycles',
'rec.sport.baseball',
'rec.sport.hockey',
'alt.atheism',
'comp.windows.x',
'misc.forsale',
'sci.crypt',
'sci.electronics',
'sci.med',
'sci.space',
'soc.religion.christian',
'talk.politics.guns',
'talk.politics.mideast',
'talk.politics.misc',
'talk.religion.misc'
]

#initialize


c = fetch_20newsgroups(subset = 'train', categories = category, shuffle = True, random_state = 42)
#print(graphics_train.target_names)
#initialize
#stop words
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download()
nltk.download('stopwords' )
stop_words_skt = text.ENGLISH_STOP_WORDS
stop_words_en = stopwords.words('english')
from string import punctuation
combined_stopwords = set.union(set(stop_words_en),set(punctuation),set(stop_words_skt))

analyzer = CountVectorizer().build_analyzer()

wnl = nltk.wordnet.WordNetLemmatizer()
def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))
def stem_rmv_punc(doc):
    return (word for word in lemmatize_sent(analyzer(doc)) if word not in combined_stopwords and not word.isdigit())
def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 
def lemmatize_sent(list_word): 
    # Text input is string, returns array of lowercased strings(words).
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word)]


min_dfs = 5 #2 or 5


#print number of documents and terms in classes
# min_df = 2 and min_df = 5
count_vect = CountVectorizer(min_df=min_dfs ,analyzer=stem_rmv_punc)#frequency count
 #tfidf transformer
#stop words

c_counts = count_vect.fit_transform(c.data).toarray()



#print(len(tf_c))
#print(len(tf_c[0]))

def tfdTotfc(c_counts, c):
    
    tf_c = [[0 for x in range(20)] for y in range(len(c_counts[0]))] 
    for i in range(len(c_counts[0])):
        sum = [0] * 20
        for j in range(len(c_counts)):
            sum[c.target[j]] += c_counts[j][i]
            #tf_c[c.target[j]][i] = sum[c.target[j]]
        tf_c[i] = sum
        #print(tf_c[i])
    return tf_c

#print(len(tfdTotfc(c_counts, c)))
#print(len(tfdTotfc(c_counts, c)[0]))
tfc = tfdTotfc(c_counts, c)
end = (time.clock() - start)
print(end)
#print number of documents and terms in classes


[nltk_data] Downloading package punkt to /Users/xukaiyuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/xukaiyuan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xukaiyuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
372.03220599999986


In [54]:
import math


#print(len(tfc))

icf = [20] * len(tfc)
for i in range(len(tfc)):
    tmp = 0
    for j in range(20):
        if(tfc[i][j] != 0):
            tmp += 1
    #print(tmp)
    icf[i] = math.log(icf[i] / tmp) + 1

tficf_ibm_hard = [0] * len(tfc[0])
tficf_mac_hard = [0] * len(tfc[0])
tficf_misc = [0] * len(tfc[0])
tficf_soc = [0] * len(tfc[0])

def cmp(a, b):
    return (a > b) - (a < b) 

def calTficf(tfc, icf, className, tficf, c):
    i = 0
    for p in range(20):
        if(cmp(className, c.target_names[p]) == 0):
            i = p
            break
    for j in range(len(tfc)):
        tficf[j] = tfc[j][i] * icf[j]
    return tficf

#for i in range(len(tfc)):
#    print(icf[i])

tficf_ibm_hard = [0] * len(tfc)
tficf_mac_hard = [0] * len(tfc)
tficf_misc = [0] * len(tfc)
tficf_soc = [0] * len(tfc)

tficf_ibm_hard = calTficf(tfc, icf, 'comp.sys.ibm.pc.hardware', tficf_ibm_hard, c)
tficf_mac_hard = calTficf(tfc, icf, 'comp.sys.mac.hardware', tficf_mac_hard, c)
tficf_misc = calTficf(tfc, icf, 'misc.forsale', tficf_misc, c)
tficf_soc = calTficf(tfc, icf, 'soc.religion.christian', tficf_soc, c)

sig_ibm = sorted(range(len(tficf_ibm_hard)), key = lambda k: tficf_ibm_hard[k])
sig_mac = sorted(range(len(tficf_mac_hard)), key = lambda k: tficf_mac_hard[k])
sig_mis = sorted(range(len(tficf_misc)), key = lambda k: tficf_misc[k])
sig_soc = sorted(range(len(tficf_soc)), key = lambda k: tficf_soc[k])
for i in range(20):
    print (c.target_names[i])


print('10 most significant terms in comp.sys.ibm.pc.hardware')
for i in range(10):
    print(count_vect.get_feature_names()[sig_ibm[len(sig_ibm) - 1 - i]])
    #print(tficf_ibm_hard[sig_ibm[len(sig_ibm) - 1 - i]])
    #print(tficf_ibm_hard[17013])

print('10 most significant terms in comp.sys.mac.hardware')
for i in range(10):
    print(count_vect.get_feature_names()[sig_mac[len(sig_mac) - 1 - i]])
    #print(tficf_mac_hard[sig_mac[len(sig_mac) - 1 - i]])

print('10 most significant terms in misc.forsale')
for i in range(10):
    print(count_vect.get_feature_names()[sig_mis[len(sig_mis) - 1 - i]])
    #print(tficf_misc[sig_mis[len(sig_mis) - 1 - i]])
    
print('10 most significant terms in soc.religion.christian')
for i in range(10):
    print(count_vect.get_feature_names()[sig_soc[len(sig_soc) - 1 - i]])
    #print(tficf_soc[sig_soc[len(sig_soc) - 1 - i]])


alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc
10 most significant terms in comp.sys.ibm.pc.hardware
scsi
drive
edu
ide
line
use
com
subject
organization
controller
10 most significant terms in comp.sys.mac.hardware
edu
line
mac
subject
organization
quadra
apple
use
scsi
problem
10 most significant terms in misc.forsale
edu
line
sale
subject
organization
new
post
com
university
offer
10 most significant terms in soc.religion.christian
god
edu
christian
jesus
say
church
subject
people
line
know
